In [ ]:
### EXTRAI AS RECEITAS DO SITE ALLRECIPES PARA O FORMATO "PRATO - INGREDIENTES" ###

In [ ]:
import requests
import pandas as pd
import time
from bs4 import BeautifulSoup
from urllib.parse import urlparse

In [ ]:
# WEBSCRAPPING DA PAGINA ALLRECIPES, PERCORRENDO TODAS AS CAMADAS PARA COLETAR LINKS DE RECEITAS

# Função para obter o conteúdo da página com cabeçalhos detalhados
def get_soup(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36',
        'Accept-Language': 'en-US,en;q=0.9',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Connection': 'keep-alive',
    }
    
    # Usando sessão para manter cookies e a consistência das requisições
    with requests.Session() as session:
        try:
            response = session.get(url, headers=headers)
            response.raise_for_status()
            return BeautifulSoup(response.text, 'html.parser')
        except requests.exceptions.RequestException as e:
            print(f"Erro ao acessar {url}: {e}")
            return None

# Função para coletar os links das categorias de receitas
def get_category_links(main_url):
    soup = get_soup(main_url)
    category_links = []

    if soup:
        category_items = soup.find_all('li', class_="comp mntl-link-list__item")
        for item in category_items:
            link = item.find('a')
            if link and link['href']:
                category_links.append(link['href'])
    return category_links

def get_recipe_links_from_subcategory(subcategory_url):
    soup = get_soup(subcategory_url)
    not_recipe_links = []
    recipe_links = []

    if soup:
        recipe_items = soup.find_all('a', class_="mntl-sc-block-universal-featured-link__link mntl-text-link button--contained-standard type--squirrel")
        for item in recipe_items:
            if item['href']:
                if '/recipe/' not in item['href'] and '-recipe-' not in item['href']:
                    not_recipe_links.append(item['href'])
                    get_recipe_links_from_subcategory(item['href'])
                else:
                    recipe_links.append(item['href'])
    return recipe_links, not_recipe_links



# Função para coletar os links das receitas dentro de uma categoria
def get_recipe_links(category_url):
    soup = get_soup(category_url)
    recipe_links = []
    not_recipe_links = []

    if soup:
        recipe_items = soup.find_all('a', class_="comp mntl-card-list-items mntl-document-card mntl-card card--image-top card card--no-image")
        for item in recipe_items:
            if item['href']:
                if '/recipe/' not in item['href'] and '-recipe-' not in item['href']:
                    new_recipe_links, new_not_recipe_links = get_recipe_links_from_subcategory(item['href'])
                    not_recipe_links.extend(new_not_recipe_links)
                    recipe_links.extend(new_recipe_links)
                else:
                    recipe_links.append(item['href'])
    return recipe_links, not_recipe_links

# URL da página principal de categorias
main_url = "https://www.allrecipes.com/recipes-a-z-6735880"

# Coleta os links das categorias
category_links = get_category_links(main_url)

# Lista para armazenar os links das receitas
all_recipe_links = []
all_not_recipe_links = []

# Itera sobre os links das categorias e coleta os links das receitas
for category_url in category_links:
    recipe_links, not_recipe_links = get_recipe_links(category_url)
    all_recipe_links.extend(recipe_links)
    all_not_recipe_links.extend(not_recipe_links)
    
    # Espera alguns segundos entre as requisições para evitar bloqueios
    time.sleep(10)

# Exibe os links das receitas coletadas
for link in all_recipe_links:
    print(link)

# for link in all_not_recipe_links:
#     print(link)

In [ ]:
# GERA ARQUIVO CSV

# Criar um DataFrame a partir da lista
df_linksRecipes = pd.DataFrame(all_recipe_links)
#df_linksNotRecipes = pd.DataFrame(all_not_recipe_links)


# Salvar a lista em um arquivo CSV sem o cabeçalho
df_linksRecipes.to_csv('ALLRECIPES_LINKS.csv', index=False, header=False)
#df_linksNotRecipes.to_csv('linkNotRecipes.csv', index=False, header=False)


In [ ]:
# GERA LISTA COM NOME DAS RECEITAS E SEUS INGREDIENTES

def extract_title_from_url(url):
    parsed_url = urlparse(url)

    path_parts = parsed_url.path

    # Se a URL contém '/recipe/'
    if '/recipe/' in parsed_url.path:
        # Retorna a última parte da URL e substitui '-' por espaço
        return path_parts.split("/")[-2].replace('-', ' ').strip()
    else:
        # Para outros casos, retorna a última parte da URL e substitui '-recipe-' por espaço
        return path_parts.split('/')[-1].split('-recipe-')[0].replace('-', ' ').strip()
        

# Função para fazer a requisição à API de ingredientes e extrair o conjunto de ingredientes
def get_ingredients_from_url(recipe_url):
    api_url = f'https://faas.schollz.com/?import=github.com/schollz/ingredients&func=IngredientsFromURL(%22{recipe_url}%22)'
    
    try:
        response = requests.get(api_url)
        response.raise_for_status()
        data = response.json()

        # Extrai os 'name' dos ingredientes
        ingredients = {ingredient['name'] for ingredient in data.get('ingredients', [])}
        return ingredients
    except requests.exceptions.RequestException as e:
        print(f"Erro ao acessar {recipe_url}: {e}")
        return set()

# Lista para armazenar os dados do dataset
dataset = []

# Itera sobre a lista de URLs de receitas
for url in all_recipe_links:
    title = extract_title_from_url(url)
    ingredients = get_ingredients_from_url(url)
    
    # Adiciona o título e os ingredientes ao dataset
    dataset.append({'prato': title, 'ingredientes': ingredients})

# Cria o DataFrame com os dados coletados
df = pd.DataFrame(dataset)

# Exibe o DataFrame
print(df)

# Se quiser salvar o dataset em CSV
df.to_csv('ALLRECIPES_PRATOS.csv', index=False)
